In [17]:
# # Loading sample image

# import torchvision.io
# import os
# data_dir = 'E:\DUK\Summer Internship\Dataset\pascal VOC\VOC2012'  # Path to the Pascal VOC dataset
# image_id = '2007_000032' 
# image_file_path = os.path.join(data_dir, 'JPEGImages', f'{image_id}.jpg')
# image = torchvision.io.read_image(image_file_path)
# image.shape

In [18]:
# # displaying 10 images sample code

# import os
# import cv2
# import xml.etree.ElementTree as ET

# # Replace these paths with your dataset paths
# voc_images_dir = 'E:\DUK\Summer Internship\Dataset\pascal VOC\JPEGImages'
# voc_annotations_dir = 'E:\DUK\Summer Internship\Dataset\pascal VOC\Annotations'

# # Get the list of image filenames
# image_filenames = os.listdir(voc_images_dir)[:10]

# for image_filename in image_filenames:
#     # Load image
#     image_path = os.path.join(voc_images_dir, image_filename)
#     image = cv2.imread(image_path)

#     # Load corresponding annotation
#     annotation_filename = image_filename.replace('.jpg', '.xml')
#     annotation_path = os.path.join(voc_annotations_dir, annotation_filename)

#     # Parse annotation XML
#     tree = ET.parse(annotation_path)
#     root = tree.getroot()

#     # Extract object labels if needed
#     object_labels = []
#     for obj in root.findall('object'):
#         obj_label = obj.find('name').text
#         object_labels.append(obj_label)

#     print(f"Image: {image_filename}, Object Labels: {object_labels}")

#     # # Display the image (you can remove this line if not needed)
#     # cv2.imshow('Image', image)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()


In [19]:
# import os
# import cv2

# # Replace these paths with your dataset paths
# voc_images_dir = 'E:\DUK\Summer Internship\Dataset\pascal VOC\JPEGImages'

# # Get the list of image filenames
# image_filenames = os.listdir(voc_images_dir)[:10]

# for image_filename in image_filenames:
#     # Load image
#     image_path = os.path.join(voc_images_dir, image_filename)
#     image = cv2.imread(image_path)

#     new_width = 800
#     new_height = 600
#     resized_image = cv2.resize(image, (new_width, new_height))

    # # Display the original and resized images (you can remove this if not needed)
    # cv2.imshow('Original Image', image)
    # cv2.imshow('Resized Image', resized_image)
    
    # # Wait for a key press and close the displayed windows
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [20]:
# image pyramid
import cv2
def create_image_pyramid(image, num_levels):
    pyramid = [image]  # Initialize the image pyramid with the original image
    temp_image = image.copy()

    for i in range(num_levels - 1):
        # Downsample the image by half using simple resizing
        temp_image = cv2.resize(temp_image, (temp_image.shape[1] // 2, temp_image.shape[0] // 2))
        pyramid.append(temp_image)

    return pyramid

In [21]:
# # Loading image, created image pyramid

# import os
# import cv2

# # Replace these paths with your dataset paths
# voc_images_dir = 'E:\DUK\Summer Internship\Dataset\pascal VOC\JPEGImages'

# # Get the list of image filenames
# image_filenames = os.listdir(voc_images_dir)[:10]
# image_pyramids = []
# for image_filename in image_filenames:
#     # Load image
#     image_path = os.path.join(voc_images_dir, image_filename)
#     image = cv2.imread(image_path)

#     new_width = 900
#     new_height = 600
#     resized_image = cv2.resize(image, (new_width, new_height))
#     temp = create_image_pyramid(resized_image, 5)
#     image_pyramids.append(temp)
    

In [16]:
# len(image_pyramids[0])

In [23]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

def extract_features(image):
    with torch.no_grad():
        resnet.eval()  # Set the model to evaluation mode
        features = resnet(image)
    return features 

resnet = models.resnet50(pretrained=True)

# Step 2: Remove the classification head (fully connected layer)
resnet = nn.Sequential(*list(resnet.children())[:-2])  # Remove the last two layers (avgpool and fc)

# Step 3: Replace global pooling with adaptive pooling
resnet.add_module('adaptive_pool', nn.AdaptiveAvgPool2d(1))  # Preserve the spatial dimensions of the feature map

# Step 4: Freeze the model's weights
for param in resnet.parameters():
    param.requires_grad = False

def process_image(image):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),  # Convert NumPy array to PIL Image
        # transforms.Resize((224, 224)),  # ResNet input size is usually 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = preprocess(image)
    image = torch.unsqueeze(image, 0)  # Add batch dimension
    return image

c:\Users\LENOVO\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LENOVO\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
# Loading image, created image pyramid, Extracting feature using resnet

import os
import cv2

# Replace these paths with your dataset paths
voc_images_dir = 'E:\DUK\Summer Internship\Dataset\pascal VOC\JPEGImages'

# Get the list of image filenames
image_filenames = os.listdir(voc_images_dir)[:10]
image_pyramids = []
for image_filename in image_filenames:
    # Load image
    image_path = os.path.join(voc_images_dir, image_filename)
    image = cv2.imread(image_path)

    new_width = 900
    new_height = 600
    resized_image = cv2.resize(image, (new_width, new_height))
    temp = create_image_pyramid(resized_image, 5)
    image_pyramids.append(temp)

full_features = []
for image_pyramid in image_pyramids:
    features_list = []
    for i, image in enumerate(image_pyramid):
        image_torch = process_image(image)  # Convert the image to PyTorch format
        features = extract_features(image_torch)
        
        # Resize the features to match the spatial dimensions of the image
        features_resized = torch.nn.functional.interpolate(features, size=(image.shape[0], image.shape[1]), mode='bilinear', align_corners=False)
        
        features_list.append(features_resized)
        print(f"Features for Level {i}: {features_resized.shape}")
    full_features.append(features_list)
    

Features for Level 0: torch.Size([1, 2048, 600, 900])
Features for Level 1: torch.Size([1, 2048, 300, 450])
Features for Level 2: torch.Size([1, 2048, 150, 225])
Features for Level 3: torch.Size([1, 2048, 75, 112])
Features for Level 4: torch.Size([1, 2048, 37, 56])


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4423680000 bytes.